# Machine Learning in Python - Group Project 1

**Due Friday, March 10th by 16.00 pm.**

*include contributors names here (such as Benjamin Chooyin, Name2, ...)*

## General Setup

In [5]:
# Add any additional libraries or submodules below

# Data libraries
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules that are necessary
import sklearn

In [6]:
# Load data
data = pd.read_csv("the_office.csv")
data = pd.DataFrame(data)

#Data Formatting
#split character column by columns instead of semicolons
for i in range(0,len(data.main_chars)):
   data.main_chars[i] = data.main_chars[i].split(';')
#same for writer column
for i in range(0,len(data.writer)):
   data.writer[i] = data.writer[i].split(';')
#Director
for i in range(0,len(data.director)):
   data.director[i] = data.director[i].split(';')

#turn season data into a string for future use
data.season = data.season.astype(str)
print(data)

/tmp/ipykernel_101/505531715.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.main_chars[i] = data.main_chars[i].split(';')
/tmp/ipykernel_101/505531715.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.writer[i] = data.writer[i].split(';')
/tmp/ipykernel_101/505531715.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.director[i] = data.director[i].split(';')


    season  episode      episode_name           director  \
0        1        1             Pilot       [Ken Kwapis]   
1        1        2     Diversity Day       [Ken Kwapis]   
2        1        3       Health Care  [Ken Whittingham]   
3        1        4      The Alliance     [Bryan Gordon]   
4        1        5        Basketball     [Greg Daniels]   
..     ...      ...               ...                ...   
181      9       19     Stairmageddon        [Matt Sohn]   
182      9       20    Paper Airplane     [Jesse Peretz]   
183      9       21  Livin' the Dream    [Jeffrey Blitz]   
184      9       22           A.A.R.M     [David Rogers]   
185      9       24            Finale       [Ken Kwapis]   

                                              writer  imdb_rating  \
0    [Ricky Gervais, Stephen Merchant, Greg Daniels]          7.6   
1                                       [B.J. Novak]          8.3   
2                                 [Paul Lieberstein]          7.9   
3  

After making sure that all the necessary libraries or submodules are uploaded here, please follow the given skeleton to create your project report. 
- Your completed assignment must follow this structure 
- You should not add or remove any of these sections, if you feel it is necessary you may add extra subsections within each (such as *2.1. Encoding*). 

**Do not forget to remove the instructions for each section in the final document.**

## 1. Introduction

*This section should include a brief introduction to the task and the data (assume this is a report you are delivering to a client).* 

- If you use any additional data sources, you should introduce them here and discuss why they were included.

- Briefly outline the approaches being used and the conclusions that you are able to draw.

## 2. Exploratory Data Analysis and Feature Engineering

*Include a detailed discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling.* 

- Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up. 
- Think carefully about whether each plot needs to be included in your final draft - your report should include figures but they should be as focused and impactful as possible.

*Additionally, this section should also implement and describe any preprocessing / feature engineering of the data.*

- Specifically, this should be any code that you use to generate new columns in the data frame `d`. All of this processing is explicitly meant to occur before we split the data in to training and testing subsets. 
- Processing that will be performed as part of an sklearn pipeline can be mentioned here but should be implemented in the following section.*

**All code and figures should be accompanied by text that provides an overview / context to what is being done or presented.**

In [7]:
#Exploratory data analysis to get a sense of the variables
print(data.head())
print(data.info)
print(data.describe())

#The data contains 186 episodes with a mean ammount of votes 2130 per episode and a mean rating of 8.25. 
#sns.pairplot(data, height = 50)
'''Many of these pairplots contain little information but there are some interesting trends. Season 8 was unpopular.
The ratings are normally distributed, The number of lines and number of words are strongly correlated so we may not need both 
variables. Total votes and imbd rating look like they could have some qaudratic correlation 
'''

  season  episode   episode_name           director  \
0      1        1          Pilot       [Ken Kwapis]   
1      1        2  Diversity Day       [Ken Kwapis]   
2      1        3    Health Care  [Ken Whittingham]   
3      1        4   The Alliance     [Bryan Gordon]   
4      1        5     Basketball     [Greg Daniels]   

                                            writer  imdb_rating  total_votes  \
0  [Ricky Gervais, Stephen Merchant, Greg Daniels]          7.6         3706   
1                                     [B.J. Novak]          8.3         3566   
2                               [Paul Lieberstein]          7.9         2983   
3                                  [Michael Schur]          8.1         2886   
4                                   [Greg Daniels]          8.4         3179   

     air_date  n_lines  n_directions  n_words  n_speak_char  \
0  2005-03-24      229            27     2757            15   
1  2005-03-29      203            20     2808            12   

'Many of these pairplots contain little information but there are some interesting trends. Season 8 was unpopular.\nThe ratings are normally distributed, The number of lines and number of words are strongly correlated so we may not need both \nvariables. Total votes and imbd rating look like they could have some qaudratic correlation \n'

In [8]:
#Code for creating unique list of main characters if needed
#Can be copied for other categorical columns too
u_characters = []
for i in range(0,len(data.main_chars)):
    for j in range(0,len(data.main_chars[i])):
        u_characters.append(data.main_chars[i][j])
u_characters= [set(u_characters)]



In [9]:
##No longer needed


#Converting the director names into a list so that they are able to operate within MultilabelBinarizer below
#Run this code chunk only once after assigning the data!! Otherwise an unhashable list error will occur
#for i in range(0,len(data.director)):
 #  data.director[i] = [data.director[i]]
#for i in range(0,len(data.director)):
#   data.writer[i] = [data.writer[i]]

In [70]:
#One hot encoding  all of the categorical columns
#Import function for multi variable one hot encoder 
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

#Directors
encoded_directors = pd.DataFrame(mlb.fit_transform(data['director']), columns= mlb.classes_)

#Writer
encoded_writers = pd.DataFrame(mlb.fit_transform(data['writer']), columns= mlb.classes_)

#Seasons as although they are an integer we dont want them to behave like an integer
encoded_seasons = pd.DataFrame(mlb.fit_transform(data['season']), columns= mlb.classes_)

#One hot encoding the main characters
encoded_characters = pd.DataFrame(mlb.fit_transform(data["main_chars"]), columns= mlb.classes_)


In [ ]:
#Removing all duplicate typos of Names
pd.set_option('display.max_rows', None)
def remove_typos(data,correct,typo):
    removed = np.zeros(shape = (186))
    data[correct] = encoded_directors[correct] + data[typo]
    data[typo] = removed
    data.drop(typo, axis = 1)

    
remove_typos(encoded_directors,'Greg Daniels', 'Greg Daneils')
remove_typos(encoded_directors,'Charles McDougal', 'Charles McDougall')

In [34]:
#Figuring out how many directors do multiple episodes
directors_count = np.mean(encoded_directors)*186
print(directors_count)

'''
directed5 = []
for i in range(0,len(directors_count)):
    if directors_count[i] < 5:
        del directors_count[i]
print(directors_count)
print('There were this many directors that did more than 5 episodes' , len(directed5))

count = np.mean(encoded_writers)*186
did5= []
for i in range(0,len(count)):
    if count[i] >= 5:
        did5.append(count[i])
print('There were this many writers that did more than 5 episodes' , len(did5))
'''



Alex Hardcastle           1.0
Amy Heckerling            1.0
Asaad Kelada              1.0
B.J. Novak                5.0
Brent Forrester           4.0
Brian Baumgartner         1.0
Bryan Cranston            1.0
Bryan Gordon              2.0
Charles McDougal          1.0
Charles McDougall         6.0
Charlie Grandy            1.0
Claire Scanlon            1.0
Claire Scanlong           1.0
Craig Zisk                1.0
Daniel Chun               1.0
Danny Leiner              1.0
David Rogers              9.0
Dean Holland              2.0
Dennie Gordon             2.0
Ed Helms                  2.0
Eric Appel                1.0
Gene Stupnitsky           2.0
Greg Daneils              1.0
Greg Daniels             14.0
Harold Ramis              4.0
J.J. Abrams               1.0
Jason Reitman             2.0
Jeffrey Blitz            11.0
Jennifer Celotta          3.0
Jesse Peretz              1.0
John Krasinski            3.0
John Scott                1.0
Jon Favreau               1.0
Joss Whedo

"\ndirected5 = []\nfor i in range(0,len(directors_count)):\n    if directors_count[i] < 5:\n        del directors_count[i]\nprint(directors_count)\nprint('There were this many directors that did more than 5 episodes' , len(directed5))\n\ncount = np.mean(encoded_writers)*186\ndid5= []\nfor i in range(0,len(count)):\n    if count[i] >= 5:\n        did5.append(count[i])\nprint('There were this many writers that did more than 5 episodes' , len(did5))\n"

In [72]:
#Removing all duplicate typos of Names
pd.set_option('display.max_rows', None)
def remove_typos(data,correct,typo):
    removed = np.zeros(shape = (186))
    data[correct] = encoded_directors[correct] + data[typo]
    data[typo] = removed
    data.drop(typo, axis = 1)

    
remove_typos(encoded_directors,'Greg Daniels', 'Greg Daneils')



15.0
Function worked as intended


"    \nprint(encoded_directors['Greg Daneils'])\nencoded_directors['Greg Daniels'] = encoded_directors['Greg Daniels'] + encoded_directors['Greg Daneils']\nencoded_directors.drop('Greg Daniels',axis = 1)\nencoded_directors['Greg Daniels']\n"

In [45]:
#Joining the One hot encoded columns with the original data frame
data_numeric = data.join([encoded_seasons,encoded_characters])
data_numeric = data_numeric.drop(['director','season', 'main_chars','episode_name','air_date','writer'], axis=1)

data_numeric = data_numeric.join(encoded_writers, lsuffix = ' Writer')
data_numeric = data_numeric.join(encoded_directors, lsuffix = ' Director')

#print(data_numeric.head())
#pd.set_option('display.max_columns', None)

(186, 60) (186, 40) (186, 133) (186, 13) (186, 17)
   episode  imdb_rating  total_votes  n_lines  n_directions  n_words  \
0        1          7.6         3706      229            27     2757   
1        2          8.3         3566      203            20     2808   
2        3          7.9         2983      244            21     2769   
3        4          8.1         2886      243            24     2939   
4        5          8.4         3179      230            49     2437   

   n_speak_char  1  2  3  4  5  6  7  8  9  Andy  Angela  Creed  Darryl  \
0            15  1  0  0  0  0  0  0  0  0     0       1      0       0   
1            12  1  0  0  0  0  0  0  0  0     0       1      0       0   
2            13  1  0  0  0  0  0  0  0  0     0       1      0       0   
3            14  1  0  0  0  0  0  0  0  0     0       1      0       0   
4            18  1  0  0  0  0  0  0  0  0     0       1      0       1   

   Dwight  Erin  Jim  Kelly  Kevin  Meredith  Michael  Oscar  Pam

## 3. Model Fitting and Tuning

In [ ]:
sns.catplot(
    data=data, x="director", y="imdb_rating", height = 5, aspect = 15, s = 20)
sns.catplot(
    data=data, x="writer", y="imdb_rating", height = 5, aspect = 15, s = 20)

*In this section you should detail your choice of model and describe the process used to refine and fit that model.*

- You are strongly encouraged to explore many different modeling methods (e.g. linear regression, regression trees, lasso, etc.) but you should not include a detailed narrative of all of these attempts. 
- At most this section should mention the methods explored and why they were rejected - most of your effort should go into describing the model you are using and your process for tuning and validatin it.

*For example if you considered a linear regression model, a classification tree, and a lasso model and ultimately settled on the linear regression approach then you should mention that other two approaches were tried but do not include any of the code or any in depth discussion of these models beyond why they were rejected. This section should then detail is the development of the linear regression model in terms of features used, interactions considered, and any additional tuning and validation which ultimately led to your final model.* 

**This section should also include the full implementation of your final model, including all necessary validation. As with figures, any included code must also be addressed in the text of the document.**

## 4. Discussion and Conclusions


*In this section you should provide a general overview of **your final model**, its **performance**, and **reliability**.* 

- You should discuss what the implications of your model are in terms of the included features, predictive performance, and anything else you think is relevant.

- This should be written with a target audience of a NBC Universal executive who is with the show and university level mathematics but not necessarily someone who has taken a postgraduate statistical modeling course. 

- Your goal should be to convince this audience that your model is both accurate and useful.

- Finally, you should include concrete recommendations on what NBC Universal should do to make their reunion episode a popular as possible.

**Keep in mind that a negative result, i.e. a model that does not work well predictively, but that is well explained and justified in terms of why it failed will likely receive higher marks than a model with strong predictive performance but with poor or incorrect explanations / justifications.**

## 5. References

*In this section, you should present a list of external sources (except the course materials) that you used during the project, if any*

- Additional data sources can be cited here, in addition to related python documentations, any other webpage sources that you benefited from